# 06_policy_recommendations

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load all outputs
o2 = pd.read_csv('02_output.csv')
o3 = pd.read_csv('03_output.csv')
o4 = pd.read_csv('04_output.csv')

merged = o4.merge(o3[['state', 'district', 'volatility']], on=['state', 'district'])
merged = merged.merge(o2[['state', 'district', 'dominance_strength', 'dominant_type']], on=['state', 'district'])

def recommend(row):
    tier = row['pressure_tier']
    typo = row['typology']
    dtype = row['dominant_type']
    
    action = "Deploy mobile enrollment vans"
    rationale = "High adult correction load + volatility indicates migrant churn."
    
    if typo == 'School-linked Surge Zone':
        action = "Launch school-based biometric drives"
        rationale = "High child ratio indicates school admission season pressure."
    elif typo == 'Correctional Overload Zone':
        action = "Set up temporary camps"
        rationale = "Adult dominance suggests high demand for updates."
    elif tier == 'Critical Infrastructure Stress':
        action = "Set up temporary camps"
        rationale = "Critical stress levels require immediate capacity expansion."
    elif row['dominance_strength'] == 'Strong Dominant':
        if dtype == 'Biometric':
             action = "Increase biometric operators"
             rationale = "Biometric heavy load requires specialized operators."
        
    return pd.Series([action, rationale])

merged[['recommended_action', 'rationale']] = merged.apply(recommend, axis=1)
out_06 = merged[['state', 'district', 'typology', 'pressure_tier', 'recommended_action', 'rationale']]
out_06.rename(columns={'typology': 'classification'}, inplace=True)
out_06.to_csv('06_output.csv', index=False)
print('Saved 06_output.csv')